In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("Fake.csv")

In [3]:
df.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [4]:
df.shape

(23481, 4)

In [5]:
df['output'] = 0

In [6]:
df.drop(['text','subject','date'], axis = 1,inplace = True)

In [7]:
df2 = pd.read_csv("True.csv")

In [8]:
df2.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [9]:
df2['output'] = 1

In [10]:
df2.drop(['text','subject','date'], axis = 1,inplace = True)

In [11]:
combined_df = pd.concat([df, df2], join='outer')

In [12]:
combined_df

,title,output
0,Donald Trump Sends Out Embarrassing New Year’...,0
1,Drunk Bragging Trump Staffer Started Russian ...,0
2,Sheriff David Clarke Becomes An Internet Joke...,0
3,Trump Is So Obsessed He Even Has Obama’s Name...,0
4,Pope Francis Just Called Out Donald Trump Dur...,0
...,...,...
21412,'Fully committed' NATO backs new U.S. approach...,1
21413,LexisNexis withdrew two products from Chinese ...,1
21414,Minsk cultural hub becomes haven from authorities,1
21415,Vatican upbeat on possibility of Pope Francis ...,1


In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, SpatialDropout1D
from tensorflow.keras.callbacks import EarlyStopping

In [15]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(df['title'])

# Convert text to sequences
X = tokenizer.texts_to_sequences(df['title'])
X = pad_sequences(X, maxlen=100)

# Splitting the dataset
X_train, X_test, y_train, y_test = train_test_split(X, df['output'], test_size=0.2, random_state=42)

# Model architecture
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=100))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Training the model
history = model.fit(X_train, y_train, epochs=10, batch_size=64, validation_split=0.1, callbacks=[early_stopping])

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Accuracy:", accuracy)

Epoch 1/10


C:\Users\diwak\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


265/265 ━━━━━━━━━━━━━━━━━━━━ 27s 93ms/step - accuracy: 0.9928 - loss: 0.0753 - val_accuracy: 1.0000 - val_loss: 3.3794e-05
Epoch 2/10
265/265 ━━━━━━━━━━━━━━━━━━━━ 24s 92ms/step - accuracy: 1.0000 - loss: 2.9789e-05 - val_accuracy: 1.0000 - val_loss: 1.5068e-05
Epoch 3/10
265/265 ━━━━━━━━━━━━━━━━━━━━ 24s 92ms/step - accuracy: 1.0000 - loss: 1.4589e-05 - val_accuracy: 1.0000 - val_loss: 9.3193e-06
Epoch 4/10
265/265 ━━━━━━━━━━━━━━━━━━━━ 24s 92ms/step - accuracy: 1.0000 - loss: 9.2395e-06 - val_accuracy: 1.0000 - val_loss: 6.5351e-06
Epoch 5/10
265/265 ━━━━━━━━━━━━━━━━━━━━ 24s 92ms/step - accuracy: 1.0000 - loss: 6.5411e-06 - val_accuracy: 1.0000 - val_loss: 4.8918e-06
Epoch 6/10
265/265 ━━━━━━━━━━━━━━━━━━━━ 26s 99ms/step - accuracy: 1.0000 - loss: 4.9317e-06 - val_accuracy: 1.0000 - val_loss: 3.8051e-06
Epoch 7/10
265/265 ━━━━━━━━━━━━━━━━━━━━ 25s 93ms/step - accuracy: 1.0000 - loss: 3.8563e-06 - val_accuracy: 1.0000 - val_loss: 3.0380e-06
Epoch 8/10
265/265 ━━━━━━━━━━━━━━━━━━━━ 24s 92ms/

In [21]:
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt

# Predict on the test data
y_pred = np.argmax(model.predict(X_test), axis=-1)

# Generate confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Check if confusion matrix has only one column
if cm.shape[1] == 1:
  print("Confusion matrix not available due to single class prediction")
else:
  # Plot confusion matrix (same as before)
  plt.figure(figsize=(4, 4))  # Adjust figure size for single class case
  plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
  plt.title('Confusion Matrix')
  plt.colorbar()
  plt.xticks([0], ['Predicted'], fontsize=14)  # Use single label for x-axis
  plt.yticks([0], ['True Label'], fontsize=14)
  plt.xlabel('Predicted Label', fontsize=14)
  plt.ylabel('True Label', fontsize=14)
  plt.text(0, 0, cm[0, 0], va='center', ha='center', fontsize=12, color='black')
  plt.show()

# Generate classification report
report = classification_report(y_test, y_pred, target_names=['Negative'], output_dict=True)  # Adjust target names
report = classification_report(y_test, y_pred, target_names=['Positive'], output_dict=True) 
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=['Positive']))


147/147 ━━━━━━━━━━━━━━━━━━━━ 3s 22ms/step
Confusion matrix not available due to single class prediction
Classification Report:
              precision    recall  f1-score   support

    Positive       1.00      1.00      1.00      4697

    accuracy                           1.00      4697
   macro avg       1.00      1.00      1.00      4697
weighted avg       1.00      1.00      1.00      4697

